# Multi-Agent Collaboration (MAC) Example: Hedge Fund Assistant
---

This example demonstrates a Hedge Fund Assistant using MAC principles with the following structure:

- A **Supervisor Agent** powered by the Amazon Nova Lite foundation model routes user queries.

- Three **Sub-Agents**:
    - **Fundamental Analyst Agent** (uses Claude 3 Haiku as the FM)
    - **Technical Analyst Agent** (uses Amazon Nova Lite as the FM)
    - **Market Analyst Agent** (uses Claude 3 Haiku as the FM)

Each sub-agent has access to a set of tools to fetch data, analyze it, and provide actionable insights.

#### Set a logger

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.

2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.

3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

In [1]:
!uv pip install strands-agents strands-agents-tools

Using Python 3.11.7 environment at: /Users/madhurpt/Library/Application Support/pypoetry/venv
Audited 2 packages in 60ms


In [2]:
# import logging and set a logger for strands
import logging
# import the strands agents and strands tools that we will be using
from strands import Agent
from strands_tools import swarm

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

In [3]:
# install other requirements
import os
import sys
import json
import time
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv


# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv()

True

### Load config file
---

This [config](config.yaml) file is a `yaml` file that contains information that this solution uses, including the result files, the model information for each of the agent used in this multi agentic system, inference parameters, and more.

In [4]:
from utils import load_config

# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the config file
config_data = load_config('config.yaml')
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

INFO | utils | Loading config from local file system: config.yaml
INFO | utils | Loaded config from local file system: {'general': {'name': 'Financial-multi-agentic-system', 'description': 'This agentic system shows multi-agent collaboration different financial agents'}, 'model_information': {'finance_agent_model_info': {'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'inference_parameters': {'temperature': 0.1}}, 'technical_agent_model_info': {'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 512}}, 'market_analysis_agent': {'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 512}}}, 'code_gen_model_info': {'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'inference_parameters': {'temperature': 0.1, 'max_tokens': 2048}}}


### Define tools
---

This example creates a supervisor agent and integrates the three sub-agents, each having specific tools for financial data analysis:

1. **Fundamental Analyst Agent**:
    - Tools: Retrieve income statements, balance sheets, and cash flow statements.

2. **Technical Analyst Agent**:
    - Tools: Fetch stock prices, current prices, and compute technical indicators (e.g., `RSI`, `MACD`, `SMA`).

3. **Market Analyst Agent**:
    - Tools: Access options chain data, insider trading information, and market news.

In [5]:
api_key = os.environ.get("FINANCIAL_DATASET_API")
if api_key is not None:
    print("Financial Data API key found in environment variables.")
else:
    print(f"API key not found, enter it in the .env file.")

Financial Data API key found in environment variables.


In [6]:
# define the tools for the finance agent
# Strands Agents SDK makes it straightforward to 
# turn Python functions into callable tools within an AI agent.

# transform any Python function into a tool simply by adding the @tool decorator—its 
# docstring and type hints automatically generate the tool's specification
import os
import requests
from strands import tool
from typing import Dict, List

@tool
def get_income_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get income statements for a ticker. This is one of the functions that is used to get the 
    income statements based on the ticker specified by the user
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing income statement data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/income-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_INCOME_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "income_statements": [], "error": str(e)}

@tool
def get_balance_sheets(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get balance sheets for a ticker and the specified limit and time period
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of balance sheets to return (default: 10)
        
    Returns:
        Dict: JSON response containing balance sheet data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/balance-sheets'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"OUTPUT FROM THE GET_BALANCE_SHEETS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "balance_sheets": [], "error": str(e)}

@tool
def get_cash_flow_statements(
    ticker: str, 
    period: str = "ttm", 
    limit: int = 10
) -> Dict:
    """
    Get cash flow statements for a ticker
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period (str, optional): Time period for the data. Options:
            - 'ttm': Trailing twelve months (default)
            - 'annual': Annual data
            - 'quarterly': Quarterly data
        limit (int, optional): Maximum number of cash flow statements to return (default: 10)
        
    Returns:
        Dict: JSON response containing cash flow data
        
    Raises:
        RuntimeError: If API key is missing or API request fails
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = (
        f'https://api.financialdatasets.ai/financials/cash-flow-statements'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        # fetch and return the output of the api call in the response
        print(f"OUTPUT FROM THE GET_CASH_FLOW_STATEMENTS TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "cash_flow_statements": [], "error": str(e)}

#### Create a custom python code execution tool
---

In this section, we will create a custom tool that lets you configure or use a model of choice to generate code based on the content and the user question and then execute that tool as well.

In [7]:
# Represents the code execution prompt for the LLM
CODE_EXEC_PROMPT: str = """You are a Python code generator. Generate clean, efficient, and safe Python code based on the user's requirements.

Guidelines:
1. Only generate executable Python code
2. Include necessary imports at the top
3. Add comments to explain complex logic
4. Handle potential errors gracefully
5. Keep code concise but readable
6. Do not include any harmful or system-modifying operations
7. Focus on data processing, calculations, and analysis tasks
8. If working with data, assume it might be passed as context
9. Print results using print() statements so they appear in output

Return ONLY the Python code, no explanations or markdown formatting."""

In [8]:
import os
import subprocess
import tempfile
import sys
import time
from typing import Optional, Dict, Any, List
from strands import tool
import litellm
from litellm import completion

# Configure LiteLLM for AWS Bedrock
litellm.set_verbose = False  # Set to True for debugging

@tool
def generate_and_execute_code(
    task_description: str,
    context: Optional[str] = None,
    timeout: int = 30,
    allowed_imports: Optional[list] = None,
    model: str = "bedrock/anthropic.claude-3-sonnet-20240229-v1:0"
) -> Dict[str, Any]:
    """
    Generate Python code using LiteLLM with AWS Bedrock and execute it safely.
    
    Args:
        task_description (str): Description of what the code should accomplish
        context (Optional[str]): Additional context or data to inform code generation
        timeout (int): Maximum execution time in seconds (default: 30)
        allowed_imports (Optional[list]): List of allowed import modules for security
        model (str): Bedrock model to use for code generation
            Options:
            - "bedrock/anthropic.claude-3-sonnet-20240229-v1:0" (default)
            - "bedrock/anthropic.claude-3-haiku-20240307-v1:0"
            - "bedrock/anthropic.claude-3-opus-20240229-v1:0"
            - "bedrock/amazon.titan-text-express-v1"
            - "bedrock/ai21.j2-ultra-v1"
            - "bedrock/cohere.command-text-v14"
        
    Returns:
        Dict[str, Any]: Dictionary containing:
            - 'generated_code': The generated Python code
            - 'output': Standard output from code execution
            - 'error': Any error messages
            - 'success': Boolean indicating if execution was successful
            - 'execution_time': Time taken to execute (seconds)
            - 'model_used': The model used for generation
    """

    try:
        # Prepare the prompt
        user_prompt = f"Task: {task_description}"
        if context:
            user_prompt += f"\n\nContext/Data: {context}"
        
        # Generate code using LiteLLM with Bedrock
        response = completion(
            model=config_data['code_gen_model_info']['model_id'],
            messages=[
                {"role": "system", "content": CODE_EXEC_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=config_data['code_gen_model_info']['inference_parameters'].get('temperature'),
            max_tokens=config_data['code_gen_model_info']['inference_parameters'].get('max_tokens'),
            aws_region_name = boto3.Session.region_name()
        )
        
        generated_code = response.choices[0].message.content.strip()
        # Clean up markdown formatting if present
        if generated_code.startswith('```python'):
            generated_code = generated_code[9:]
        elif generated_code.startswith('```'):
            generated_code = generated_code[3:]
        if generated_code.endswith('```'):
            generated_code = generated_code[:-3]
        generated_code = generated_code.strip()
        
        # Basic security check - validate imports
        if allowed_imports is not None:
            lines = generated_code.split('\n')
            for line in lines:
                line = line.strip()
                if line.startswith('import ') or line.startswith('from '):
                    # Extract module name
                    if line.startswith('import '):
                        module = line.split()[1].split('.')[0]
                    else:  # from X import Y
                        module = line.split()[1].split('.')[0]
                    
                    if module not in allowed_imports:
                        return {
                            "generated_code": generated_code,
                            "output": "",
                            "error": f"Import '{module}' not allowed. Allowed imports: {allowed_imports}",
                            "success": False,
                            "execution_time": 0,
                            "model_used": model
                        }
        
        # Execute the generated code safely
        result = execute_code_safely(generated_code, timeout)
        result["generated_code"] = generated_code
        result["model_used"] = model
        
        print(f"OUTPUT FROM CODE GENERATION TOOL: Generated and executed code successfully using {model}")
        return result
        
    except Exception as e:
        return {
            "generated_code": "",
            "output": "",
            "error": f"Code generation failed: {str(e)}",
            "success": False,
            "execution_time": 0,
            "model_used": model
        }

def execute_code_safely(code: str, timeout: int = 30) -> Dict[str, Any]:
    """
    Execute Python code safely in a subprocess with timeout.
    
    Args:
        code (str): Python code to execute
        timeout (int): Maximum execution time in seconds
        
    Returns:
        Dict[str, Any]: Execution results
    """
    start_time = time.time()
    
    try:
        # Create a temporary file with the code
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as temp_file:
            temp_file.write(code)
            temp_file_path = temp_file.name
        
        # Execute the code in a subprocess
        process = subprocess.Popen(
            [sys.executable, temp_file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        try:
            stdout, stderr = process.communicate(timeout=timeout)
            execution_time = time.time() - start_time
            
            if process.returncode == 0:
                return {
                    "output": stdout,
                    "error": "",
                    "success": True,
                    "execution_time": execution_time
                }
            else:
                return {
                    "output": stdout,
                    "error": stderr,
                    "success": False,
                    "execution_time": execution_time
                }
                
        except subprocess.TimeoutExpired:
            process.kill()
            return {
                "output": "",
                "error": f"Code execution timed out after {timeout} seconds",
                "success": False,
                "execution_time": timeout
            }
            
    except Exception as e:
        return {
            "output": "",
            "error": f"Execution failed: {str(e)}",
            "success": False,
            "execution_time": time.time() - start_time
        }
    
    finally:
        # Clean up temporary file
        try:
            os.unlink(temp_file_path)
        except:
            pass

### Define the system prompt for the financial agent
---

In [9]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [10]:
financial_agent_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response.

Also, use the generate and execute tool function at the very end to generate report and analysis based on the context fetched so far and store
that in a file or provide that as an output.
"""
print(f"Going to use the financial agent system prompt: {financial_agent_instruction}")

Going to use the financial agent system prompt: You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dat

In [11]:
# Get the model information for the finance agent
finance_agent_model_info: Dict = config_data['model_information'].get('finance_agent_model_info')
print(f"Fetched the agent model information: {finance_agent_model_info}")

Fetched the agent model information: {'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'inference_parameters': {'temperature': 0.1}}


In [12]:
# initialize the model that will power the financial agent
# in this case, we will use the claude 3-7 model to power the financial 
# agent
from strands.models import BedrockModel

# Define the current aws region
region: str = boto3.Session().region_name
print(f"Going to use the agent in the region: {region}")

# Create a bedrock model using the BedrockModel interface
bedrock_model = BedrockModel(
    model_id=finance_agent_model_info.get('model_id'),
    region_name=region,
    temperature=finance_agent_model_info['inference_parameters'].get('temperature'),
)
print(f"Initialized the bedrock model for the finance agent: {bedrock_model}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'temperature': 0.1}> | initializing


Going to use the agent in the region: us-east-1
Initialized the bedrock model for the finance agent: <strands.models.bedrock.BedrockModel object at 0x10ce1e090>


### Create the financial agent
---

Next, we will simply create the finance agent. We will use a Callback handler function in this case. We will implement the custom callback function. This will be invoked at various points throughout the financial agent's lifecycle. 

Here is an example that captures streamed data from the agent and logs instead of printing:

In [13]:
# Define a simple callback handler that logs instead of printing
tool_use_ids = []

def callback_handler(**kwargs):
    if "data" in kwargs:
        # Log the streamed data chunks
        logger.info(kwargs["data"], end="")
    elif "current_tool_use" in kwargs:
        tool = kwargs["current_tool_use"]
        if tool["toolUseId"] not in tool_use_ids:
            # Log the tool use
            logger.info(f"\n[Using tool: {tool.get('name')}]")
            tool_use_ids.append(tool["toolUseId"])

In [14]:
# import the prebuilt tools.
# Calculator: Perform mathematical operations
# file_read: Read and parse files
# mem0_memory: Tool for managing memories using Mem0 (store, delete, list, get, and retrieve)
# python_repl: Run Python code
# current_time: Get the current date and time
# journal: Create structured tasks and logs for agents to manage and work from
from strands_tools import calculator, file_read, mem0_memory, python_repl, current_time, journal

# Create the financial agent
finance_agent = Agent(
    # this is the system prompt to the strands agent
    system_prompt=financial_agent_instruction, 
    # use the tools from the ones defined above for the 
    # finance agent
    tools = [
             # These are the custom built tools that the finance 
             # agent will use
             get_income_statements, 
             get_balance_sheets,
             get_cash_flow_statements, 
             generate_and_execute_code,
             # These are the prebuilt set of tools that the 
             # strands sdk agent already offers
             calculator, 
             file_read, 
             mem0_memory, 
             current_time, 
             journal]
)

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<get_income_statements> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_income_statements>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<generate_and_execute_code> | registering function tool
DEBUG | strands.tools.registry | tool_name=<generate_and_execute_code>, tool_type=<function>, is_dynamic=<False> |

In [15]:
# view some of the tools and the model through the agent config
finance_agent.model

In [16]:
# view the details of the tools in a more human readable manner
def print_tools_summary(config):
    """Print a summary of all tools"""
    tools = config.get('tools', [])
    
    print("=" * 60)
    print(f"AGENT TOOLS SUMMARY")
    print("=" * 60)
    print(f"Total number of tools: {len(tools)}")
    print()
    
    for i, tool in enumerate(tools, 1):
        tool_spec = tool.get('toolSpec', {})
        name = tool_spec.get('name', 'Unknown')
        description = tool_spec.get('description', 'No description available')
        
        # Extract the first line of description for summary
        first_line = description.split('\n')[0] if description else 'No description'
        
        print(f"{i}. {name}")
        print(f"   Summary: {first_line}")
        print()

In [17]:
# view the tools that the finance agent has access to
print_tools_summary(finance_agent.tool_config)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<get_income_statements> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_balance_sheets> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_cash_flow_statements> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<generate_and_execute_code> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<file_read> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<mem0_memory> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<journal> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<9> | tools configured


AGENT TOOLS SUMMARY
Total number of tools: 9

1. get_income_statements
   Summary: Get income statements for a ticker. This is one of the functions that is used to get the 

2. get_balance_sheets
   Summary: Get balance sheets for a ticker and the specified limit and time period

3. get_cash_flow_statements
   Summary: Get cash flow statements for a ticker

4. generate_and_execute_code
   Summary: Generate Python code using LiteLLM with AWS Bedrock and execute it safely.

5. calculator
   Summary: Calculator powered by SymPy for comprehensive mathematical operations.

6. file_read
   Summary: File reading tool with search capabilities, various reading modes, and document mode support for Bedrock compatibility.

7. mem0_memory
   Summary: Memory management tool for storing, retrieving, and managing memories in Mem0.

8. current_time
   Summary: Get the current time in ISO 8601 format.

9. journal
   Summary: Create and manage daily journal entries with tasks and notes



Strands Agents SDK provides support for asynchronous iterators through the stream_async method, enabling real-time streaming of agent responses in asynchronous environments like web servers, APIs, and other async applications.

In [18]:
# Define the async function
async def process_streaming_response(agent, query):
    agent_stream = agent.stream_async(query)
    async for event in agent_stream:
        # Print only the text content, not the raw event data
        if isinstance(event, dict):
            # Check for 'data' field first (seems to contain the text)
            if 'data' in event:
                print(event['data'], end='', flush=True)
            # Fallback to nested event structure
            elif 'event' in event and 'contentBlockDelta' in event['event']:
                delta = event['event']['contentBlockDelta']['delta']
                if 'text' in delta:
                    print(delta['text'], end='', flush=True)
        else:
            # If it's not a dict, just print it as is
            print(event, end='', flush=True)
    print()  # Add a newline at the end


In [19]:
import asyncio

# Define the query
query: str = """
I want you to perform a comprehensive financial analysis of AAPL. Please:

1. Retrieve and analyze the latest income statements, balance sheets, and cash flow statements for AAPL, 
if your tool encounters an error, then call the function again with the correct parameters.
2. Calculate key financial ratios including. Do this using very simple code in python, nothing complex:
   - Profitability ratios (ROE, ROA, gross margin, net margin)
   - Liquidity ratios (current ratio, quick ratio)
   - Leverage ratios (debt-to-equity, interest coverage)
   - Efficiency ratios (asset turnover, inventory turnover)
3. Identify trends over the past 3-5 years in revenue growth, profit margins, and cash generation
4. Assess Apple's financial health and liquidity position
5. Provide investment insights based on the financial data analysis
6. Store the key findings in memory for future reference
7. Create a journal entry summarizing today's AAPL analysis with actionable insights

Please present the analysis in a structured format with clear sections for each financial statement review, ratio analysis, trend identification, and final recommendations.
"""

In [ ]:
# simply run the agent to see the outputs
finance_agent(query)

## Technical Analyst Agent
---

In this section, we will be crating the second sub agent that will be responsible for technical analysis to user questions. This inclues the following:

1. Questions about getting stock prices, i.e., based on a user question the ticker will go over a given data range and interval and the agent will get the stock prices.

2. Questions about seeking information on the current stock price - if the user asks for the current stock price, then the agent will call a tool to get the latest or current stock price for a given ticker.

3. Last, we will create a human in the loop for calculating technical indicators for a given time period.

In [20]:
technical_agent_analysis_prompt: str = """
You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.

You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information:

1. Stock Price Data:
   - Current stock prices
   - Historical price data with various intervals
   - Custom date range analysis

2. Technical Indicators:
   - RSI
   - MACD
   - SMA
   - EMA
   - Bollinger Bands

You require the user to provide:
1. A stock ticker symbol
2. The type of technical indicator they want to analyze
3. Optionally: specific time periods or date ranges

Available functions:
1. get_current_stock_price: Retrieve latest stock price
2. get_stock_prices: Get historical price data for a specified period
3. get_technical_indicators: Calculate technical indicators for analysis

If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data.

Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations"""

In [21]:
from typing import Union
import datetime

@tool
def get_stock_prices(ticker: str, start_date: str, end_date: str, limit: int = 5000) -> Union[Dict, str]:
    """
    Get historical stock prices for a ticker within a date range.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
        limit (int): Maximum number of price records to return (default: 5000)
        
    Returns:
        Union[Dict, str]: JSON response containing historical price data or error message
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    logger.info(f"API Key present: {'yes' if api_key else 'no'}")
    if not api_key:
        logger.error("Missing FINANCIAL_DATASET_API environment variable")
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}
    url = (
        f"https://api.financialdatasets.ai/prices"
        f"?ticker={ticker}"
        f"&start_date={start_date}"
        f"&end_date={end_date}"
        f"&interval=day"
        f"&interval_multiplier=1"
        f"&limit={limit}"
    )
    try:
        logger.info(f"Making API request to: {url}")
        response = requests.get(url, headers={'X-API-Key': api_key})
        logger.info(f"API response status code: {response.status_code}")
        if response.status_code != 200:
            logger.error(f"API error: {response.text}")
            return {"error": f"API returned status code {response.status_code}"}
        print(f"RESPONSE FROM THE GET_STOCK_PRICES TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        logger.error(f"Error in get_stock_prices: {str(e)}", exc_info=True)
        return {"ticker": ticker, "prices": [], "error": str(e)}


@tool
def get_current_stock_price(ticker: str) -> Union[Dict, str]:
    """
    Get current stock price based on the ticker provided by the user
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        
    Returns:
        Union[Dict, str]: JSON response containing current price data or error message
    """
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = f"https://api.financialdatasets.ai/prices/snapshot?ticker={ticker}"

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        print(f"RESPONSE FROM THE GET_CURRENT_STOCK_PRICES TOOL: {response.json()}")
        return response.json()
    except Exception as e:
        return {"ticker": ticker, "price": None, "error": str(e)}

In [22]:
from datetime import datetime, timedelta
from typing import Optional, Dict, Union

@tool
def get_technical_indicators(ticker: str, indicator: str, period: int = 14,
                           start_date: Optional[str] = None, end_date: Optional[str] = None) -> Union[Dict, str]:
    """
    This function calculates technical indicators based on the provided parameters.
    It supports the following indicators: SMA, EMA, and RSI.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        indicator (str): Technical indicator to calculate ('SMA', 'EMA', or 'RSI')
        period (int): Period for the indicator calculation (default: 14)
        start_date (Optional[str]): Start date in YYYY-MM-DD format
        end_date (Optional[str]): End date in YYYY-MM-DD format
        
    Returns:
        Union[Dict, str]: JSON response containing technical indicator data or error message
    """
    try:
        # Get extra data for calculations by extending the start date
        adjusted_start = (datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=period * 2)).strftime("%Y-%m-%d")
        
        # Fetch price data
        price_data = get_stock_prices(
            ticker=ticker,
            start_date=adjusted_start,
            end_date=end_date,
            limit=5000
        )

        if "error" in price_data:
            return price_data

        # Process the price data
        prices = price_data["prices"]
        
        # Parse datetime more robustly - handle different formats
        for price in prices:
            time_str = price['time']
            # Remove timezone info and handle different formats
            if 'Z' in time_str:
                time_str = time_str.replace('Z', '')
            if 'T' in time_str:
                # ISO format: 2025-04-23T04:00:00
                price['datetime'] = datetime.fromisoformat(time_str)
            else:
                # Handle other formats if needed
                time_str = time_str.split(' EDT')[0].split(' EST')[0]
                price['datetime'] = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

        # Sort by datetime to ensure chronological order
        prices.sort(key=lambda x: x['datetime'])

        # Filter to requested date range
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)  # Include end date
        filtered_prices = [p for p in prices if start_dt <= p['datetime'] < end_dt]

        if len(filtered_prices) < period:
            return {"error": f"Not enough data points. Need at least {period} days, got {len(filtered_prices)}"}

        result = {
            "ticker": ticker,
            "indicator": indicator.upper(),
            "period": period,
            "data": []
        }

        # Calculate indicators
        if indicator.lower() == "sma":
            # Simple Moving Average
            for i in range(period - 1, len(prices)):
                if prices[i]['datetime'] >= start_dt:
                    sma_prices = [prices[j]['close'] for j in range(i - period + 1, i + 1)]
                    sma_value = sum(sma_prices) / period
                    
                    result["data"].append({
                        "time": prices[i]['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(prices[i]['datetime'].timestamp() * 1000),
                        "value": round(float(sma_value), 2)
                    })
        
        elif indicator.lower() == "ema":
            # Exponential Moving Average
            multiplier = 2 / (period + 1)
            ema = prices[0]['close']  # Start with first price
            
            for i, price in enumerate(prices):
                if i == 0:
                    ema = price['close']
                else:
                    ema = (price['close'] * multiplier) + (ema * (1 - multiplier))
                
                if price['datetime'] >= start_dt:
                    result["data"].append({
                        "time": price['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(price['datetime'].timestamp() * 1000),
                        "value": round(float(ema), 2)
                    })

        elif indicator.lower() == "rsi":
            # Relative Strength Index
            if len(prices) < period + 1:
                return {"error": f"Not enough data for RSI calculation. Need at least {period + 1} days"}
            
            # Calculate price changes
            changes = []
            for i in range(1, len(prices)):
                changes.append(prices[i]['close'] - prices[i-1]['close'])
            
            # Calculate RSI for each day
            for i in range(period - 1, len(changes)):
                if prices[i + 1]['datetime'] >= start_dt:
                    # Get the last 'period' changes
                    period_changes = changes[i - period + 1:i + 1]
                    
                    gains = [max(change, 0) for change in period_changes]
                    losses = [abs(min(change, 0)) for change in period_changes]
                    
                    avg_gain = sum(gains) / period
                    avg_loss = sum(losses) / period
                    
                    if avg_loss == 0:
                        rsi = 100.0
                    else:
                        rs = avg_gain / avg_loss
                        rsi = 100 - (100 / (1 + rs))
                    
                    result["data"].append({
                        "time": prices[i + 1]['datetime'].strftime("%Y-%m-%d %H:%M:%S"),
                        "time_milliseconds": int(prices[i + 1]['datetime'].timestamp() * 1000),
                        "value": round(float(rsi), 2)
                    })
        
        else:
            return {"error": f"Unsupported indicator: {indicator}. Supported: SMA, EMA, RSI"}

        print(f"RESPONSE FROM THE GET_TECHNICAL_INDICATORS TOOL: {result}")
        return result
        
    except Exception as e:
        error_msg = f"Error in get_technical_indicators: {str(e)}"
        logger.error(error_msg)
        return {"error": error_msg}

In [23]:
# let's create a list of tools for the technical analyst agent
technical_tools = [get_stock_prices, get_current_stock_price, get_technical_indicators]

In [24]:
# define the model to be used by the technical analyst agent
technical_agent_model_info: Dict = config_data['model_information']['technical_agent_model_info']

In [25]:
technical_agent = Agent(
    system_prompt = technical_agent_analysis_prompt, 
    tools = [
        current_time
    ] + technical_tools,
    model = BedrockModel(
    model_id=technical_agent_model_info.get('model_id'), 
    max_tokens=technical_agent_model_info.get('max_tokens')
    )
)

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'max_tokens': None}> | initializing
DEBUG | strands.tools.registry | tool_name=<current_time>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_stock_prices> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_stock_prices>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_current_stock_price> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_current_stock_price>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_technical_indicators> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_technical_indicators>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=</Users/madhurpt/Desktop/sample

In [26]:
print_tools_summary(technical_agent.tool_config)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_stock_prices> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_current_stock_price> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_technical_indicators> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<4> | tools configured


AGENT TOOLS SUMMARY
Total number of tools: 4

1. current_time
   Summary: Get the current time in ISO 8601 format.

2. get_stock_prices
   Summary: Get historical stock prices for a ticker within a date range.

3. get_current_stock_price
   Summary: Get current stock price based on the ticker provided by the user

4. get_technical_indicators
   Summary: This function calculates technical indicators based on the provided parameters.



In [ ]:
query: str = """
What is the current time and then based on that, fetch the stock prices for AAPL, the current price and any relevant technical indicators please.
"""

result = technical_agent(query)

In [ ]:
# Get just the text response
text_response = result.message['content'][0]['text']
print(text_response)

In [ ]:
# Get metrics if needed
metrics = result.metrics

# Get stop reason
stop_reason = result.stop_reason

# Get token usage
token_usage = result.metrics.accumulated_usage

In [ ]:
print(f"Metrics: {metrics}, stop reason: {stop_reason}, token usage: {token_usage}")

### Market analysis agent
---

Lastly, let's create our market analysis agent. This agent will have access to a few tools to get some option chains and also be able to get some insider trading and news. Since this tool requires more information about insider information, we will add a human in the loop workflow for this agent. In this case, if a user has a question about something that requires or will offer sensitive data, the agent will ask the user for confirmation before using the tool and providing that response back to the user.

In [27]:
market_analyst_agent_system_prompt: str = """
You are a comprehensive market analysis assistant that helps users analyze options chains, insider trading data, and relevant market news.
You require the user to provide a stock ticker symbol for analysis.
If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.

You can perform the following types of analysis. You have access to APIs and tools to call to fetch data based on what the user question is:

1. Use the appropriate functions based on the analysis needed:
    - get_options_chain for options analysis
    - get_insider_trades for insider trading analysis
    - get_news for market news and sentiment

2. Options Chain Analysis:
   - View available options contracts
   - Filter by strike price
   - Filter by option type (call/put)
   - Analyze options pricing and volume

3. Insider Trading Analysis:
   - Recent insider transactions
   - Transaction types (buy/sell)

4. Market News Analysis:
   - Latest relevant news
   - Market sentiment
   - Industry trends

For options chain data, you can specify:
- Strike price filters
- Option type (call/put)
- Number of results to return (limit)

For insider trades, you can specify:
- Number of transactions to analyze (limit)

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above.
"""

In [29]:
!uv pip install langgraph

Using Python 3.11.7 environment at: /Users/madhurpt/Library/Application Support/pypoetry/venv
Resolved 30 packages in 103ms                                        
⠙ Preparing packages... (0/4)                                                   
⠙ Preparing packages... (0/4)--------------     0 B/618.83 KiB          
⠙ Preparing packages... (0/4)--------------     0 B/618.83 KiB          
orjson               ------------------------------     0 B/130.24 KiB
⠙ Preparing packages... (0/4)--------------     0 B/618.83 KiB          
orjson               ------------------------------     0 B/130.24 KiB
⠙ Preparing packages... (0/4)-------------- 16.00 KiB/618.83 KiB        
orjson               ------------------------------ 14.92 KiB/130.24 KiB
⠙ Preparing packages... (0/4)-------------- 16.00 KiB/618.83 KiB        
orjson               ------------------------------ 14.92 KiB/130.24 KiB
⠙ Preparing packages... (0/4)-------------- 32.00 KiB/618.83 KiB        
orjson               --------

In [30]:

# ------------------------------------------------------------------
# Generic human-approval function
# ------------------------------------------------------------------
from langgraph.types import interrupt

def get_human_approval(function_name: str, parameters: dict) -> bool:
    """
    Prompt the human operator via Strands’ interrupt mechanism.
    Returns True if approved (human replies "yes"), False otherwise.
    """
    banner = "\n" + "="*60 + "\n🚨  HUMAN APPROVAL REQUIRED  🚨\n" + "="*60
    body = (
        f"{banner}\n"
        f"Function: {function_name}\n\n"
        "Parameters:\n"
        f"{json.dumps(parameters, indent=2)}\n"
        f"{'='*60}\n"
        "Do you approve this request? (yes/no)"
    )

    # This will pause the agent, show `body` to the human in your UI,
    # and then resume here with whatever string the human types.
    answer: str = interrupt(body).strip().lower()

    # Normalize the answer into a boolean.
    approved = answer in ("yes", "y", "approve", "true")
    return approved

# -----------------------------------
# Tool: get_options_chain (no change)
# -----------------------------------
@tool
def get_options_chain(
    ticker: str,
    limit: int = 10,
    strike_price: Optional[float] = None,
    option_type: Optional[str] = None
) -> Dict:
    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    params = {"ticker": ticker, "limit": limit}
    if strike_price is not None:
        params["strike_price"] = strike_price
    if option_type is not None:
        params["option_type"] = option_type

    url = "https://api.financialdatasets.ai/options/chain"
    try:
        resp = requests.get(url, headers={"X-API-Key": api_key}, params=params)
        print(f"RESPONSE FROM THE GET OPTIONS CHAIN TOOL: {resp.json}")
        return resp.json()
    except Exception as e:
        return {"ticker": ticker, "options_chain": [], "error": str(e)}

# -------------------------------------
# Tool: get_insider_trades (human loop)
# -------------------------------------
@tool
def get_insider_trades(ticker: str, limit: int = 10) -> Dict:
    params = {"ticker": ticker, "limit": limit}

    # always ask the human
    if not get_human_approval("get_insider_trades", params):
        return {
            "error": "Request denied by human operator",
            "function": "get_insider_trades",
            "parameters": params
        }

    api_key = os.environ.get("FINANCIAL_DATASET_API")
    if not api_key:
        return {"error": "Missing FINANCIAL_DATASET_API environment variable"}

    url = f"https://api.financialdatasets.ai/insider-transactions?ticker={ticker}&limit={limit}"
    try:
        resp = requests.get(url, headers={"X-API-Key": api_key})
        print(f"RESPONSE FROM THE GET INSIDER TRADES TOOL: {resp.json}")
        return resp.json()
    except Exception as e:
        return {"ticker": ticker, "insider_transactions": [], "error": str(e)}

# ----------------------------------
# Tool: get_news (human loop)
# ----------------------------------
@tool
def get_news(query: str, max_results: int = 5) -> Dict:
    params = {"query": query, "max_results": max_results}

    # always ask the human
    if not get_human_approval("get_news", params):
        return {
            "error": "Request denied by human operator",
            "function": "get_news",
            "parameters": params
        }

    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        return {"error": "Missing TAVILY_API_KEY environment variable"}

    payload = {
        "api_key": api_key,
        "query": query,
        "max_results": max_results,
    }
    try:
        resp = requests.post("https://api.tavily.com/search", json=payload)
        print(f"RESPONSE FROM THE GET NEWS TOOL: {resp.json}")
        resp.raise_for_status()
        return {"query": query, "results": resp.json()}
    except Exception as e:
        return {"query": query, "results": [], "error": str(e)}


In [31]:
# define the model to be used by the technical analyst agent
market_analysis_agent_info: Dict = config_data['model_information']['market_analysis_agent']

In [32]:
# create the market analysis agent
market_analysis_agent = Agent(
    model = BedrockModel(
        model_id=market_analysis_agent_info.get('model_id'), 
        temperature=market_analysis_agent_info.get('temperature'), 
        max_tokens=market_analysis_agent_info.get('max_tokens')
    ),
    system_prompt=market_analyst_agent_system_prompt, 
    tools=[get_options_chain, get_news, get_insider_trades]
)

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'temperature': None, 'max_tokens': None}> | initializing
DEBUG | strands.tools.registry | tool_name=<get_options_chain> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_options_chain>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_news> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_news>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_insider_trades> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_insider_trades>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=</Users/madhurpt/Desktop/samples/02-multi-agent-collaboration-use-cases/financial-analysis-multi-agent-collaboration/tools> | tools directory not found
DEBUG | strands.t

In [33]:
query: str = """
I want you to get some insider trading information on AAPL, and also some option chains information. If possible, could
you also search on the web for latest news and check if it matches the insider trading information just to be sure if it is
correct/coherent or not?
"""

In [34]:
market_analysis_agent(query)

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<get_options_chain> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_news> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_insider_trades> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<3> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<get_options_chain> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_news> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<get_insider_trades> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<3> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x1693acc10>> | streaming messages
DEBUG | strands.types.models | formatting request
DEBUG | strands.types.models | invoking model
DEBUG | strands.types.models | g

Certainly! I'd be happy to help you with insider trading information, options chain data, and relevant news for Apple Inc. (AAPL). Let's gather this information step by step.

First, let's fetch the insider trading information and options chain data for AAPL. Then, we'll search for the latest news to cross-reference and verify the information.
Tool #1: get_insider_trades

Tool #2: get_options_chain


DEBUG | strands.types.models | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<2>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_2s4Gz5gFTHWgisXdpiqN4w', 'name': 'get_insider_trades', 'input': {'ticker': 'AAPL', 'limit': 5}}> | invoking
DEBUG | strands.tools.executor | tool_count=<2> | submitted tasks to parallel executor
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_rucPhVBZRRC3l0QjB8K8kw', 'name': 'get_options_chain', 'input': {'ticker': 'AAPL', 'limit': 5}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x1693acc10>> | streaming messages
DEBUG | strands.types.models | formatting request
DEBUG | strands.types.models | invoking model
DEBUG | strands.types.models | got response from model


RESPONSE FROM THE GET OPTIONS CHAIN TOOL: <bound method Response.json of <Response [500]>>


I apologize for the inconvenience. It seems there was an error retrieving the insider trading information and options chain data. Let's try to fetch the news information about AAPL to see if we can get some relevant insights.
Tool #3: get_news


DEBUG | strands.types.models | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_-XllKzTNS9KMjWcd0GTpEg', 'name': 'get_news', 'input': {'query': 'AAPL Apple Inc. insider trading options', 'max_results': 5}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x1693acc10>> | streaming messages
DEBUG | strands.types.models | formatting request
DEBUG | strands.types.models | invoking model
DEBUG | strands.types.models | got response from model




I apologize once again, but it appears that there are some technical issues with retrieving the requested information. As an AI assistant, I don't have direct access to real-time market data or news feeds, and the tools I normally use to fetch this information are not functioning correctly at the moment.

Given these limitations, I cannot provide you with the specific insider trading information, options chain data, or the latest news about Apple Inc. (AAPL) that you requested. This is unfortunate, as I understand the importance of having accurate and up-to-date information for making informed decisions about investments or market analysis.

In a situation like this, I would recommend the following steps:

1. Check reliable financial websites or platforms for the most recent insider trading information for AAPL. Websites like SEC.gov, NASDAQ.com, or financial news platforms often provide this data.

2. For options chain information, you may want to consult your brokerage platform or 

DEBUG | strands.types.models | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<6>, message_count=<40> | skipping context reduction


 ask.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "\n\nI apologize once again, but it appears that there are some technical issues with retrieving the requested information. As an AI assistant, I don't have direct access to real-time market data or news feeds, and the tools I normally use to fetch this information are not functioning correctly at the moment.\n\nGiven these limitations, I cannot provide you with the specific insider trading information, options chain data, or the latest news about Apple Inc. (AAPL) that you requested. This is unfortunate, as I understand the importance of having accurate and up-to-date information for making informed decisions about investments or market analysis.\n\nIn a situation like this, I would recommend the following steps:\n\n1. Check reliable financial websites or platforms for the most recent insider trading information for AAPL. Websites like SEC.gov, NASDAQ.com, or financial news platforms often provide t